# Introduction
In notebook 47 I added the vitamin B12 requirement into the biomass reaction. However I noticed that we should revisit the vitamin B12 biosynthesis as it doesnt seem to match comments made about it in the thesis. 

I opened issue #92 to discuss how to incoporate this into the model. Based on information in the thesis, provided by martyn and found in literature (e.g. Moore et al. 2012, doi:10.1042/BST20120066) we've come to the conclusion that our strain likely uses a hybrid (oxygen-independent) mechanism. The upstream part of the pathway comes from the anaerobic pathway. At the step of precorrin-5, E.C. 4.99.1.3 is used to incorporate the cobalt ion into the molecule. Then the rest of the pathway follows the aerobic part to yield cobyrinate.

Here I will make sure that these parts are incorporated properly in the model, and provide some rational for some decisions made with specific steps.

I will work top-down, starting from precorrin 2.

In [21]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [22]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

PC20M: from precorrin 2 (shcl_c) to Precorrin 3A via the CobI protein.

The next step should convert precorrin 3A to Precorrin 4. In the classical aerobic pathway, this is done via CobG and CobJ, however it seems these enzymes are not present in our strain. These two enzymes are resposible for the oxygen dependent ring contraction. 

Alternatively, our strain does have the CbiH enzyme annotated. This is the anaerobic equivalent that causes ring-contraction. It is also hypothesized that the CbiH cannot discriminate between cobalt-factor III and cobalt-precorrin-3. So it may be that the CbiH enzyme in our strain can perform this conversion without the necessity of oxygen. This enzyme is generally understudied and its function not well understood, and so this conversion will be added.

At the same time, as checking the PRE3AOR (CobG) and PC17M (CobJ) reactions, they do not seem to be present in our strain. Therefore to prevent them from carrying incorrect flux, they should be removed. 

Note: with this hypothesis, the structure of the precorrin-4 will be slightly different than would be traditionally considered, so i will name it precorrin4a. just to indicate it is a bit different. This difference gets corrected with the cobalt gets added to the molecule. (i.e. Co-precorrin5A is formed)

In [23]:
model.metabolites.pre4_c.id = 'pre4a_c'

In [24]:
model.metabolites.pre4a_c.formula = 'C44H52N4O16'

In [25]:
model.metabolites.pre4a_c.notes['NOTES'] = 'Slightly different molecule as a result of hybrid pathway'

In [26]:
del model.metabolites.pre4a_c.annotation['chebi']
del model.metabolites.pre4a_c.annotation['kegg.compound']
del model.metabolites.pre4a_c.annotation['metanetx.chemical']

In [27]:
#add reaction
model.add_reaction(Reaction(id='PRE3MT', name = 'cobalt-factor III methyltransferase'))

In [28]:
model.reactions.PRE3MT.annotation['sbo']= 'SBO:0000176'
model.reactions.PRE3MT.annotation['kegg.reaction']= 'R11580'
model.reactions.PRE3MT.annotation['ec-code']= '2.1.1.272'

In [29]:
model.groups.get_by_id('00860 - Porphyrin and chlorophyll metabolism').add_members(model.reactions.PRE3MT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



In [30]:
model.reactions.PRE3MT.add_metabolites({
    model.metabolites.amet_c:-1,
    model.metabolites.pre3a_c:-1,
    model.metabolites.ahcys_c:1,
    model.metabolites.pre4a_c:1,
    model.metabolites.h_c:1
})

In [31]:
#remove the two reactions which should not be present

In [32]:
model.remove_reactions(model.reactions.PRE3AOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [33]:
model.remove_reactions(model.reactions.PC17M)

Now that we have made the slightly modified precorrin4'A', this can be converted into precorrin 5, via the cobM enzyme. Here again, due to the ring cyclization in the precorrin 4'A', I'll change the precorrin 5 metabolite to precorrin5'A' aswell, and add a note to this metabolite so people are aware.

The reaction is the PC11M reaction, which is still correct and can be left as is.

In [34]:
model.metabolites.pre5_c.id = 'pre5a_c'

In [35]:
model.metabolites.pre5a_c.notes['NOTES'] =  'Slightly different molecule as a result of hybrid pathway'

In [36]:
del model.metabolites.pre5a_c.annotation['chebi']
del model.metabolites.pre5a_c.annotation['kegg.compound']
del model.metabolites.pre5a_c.annotation['metanetx.chemical']

In [37]:
model.metabolites.pre5a_c.formula = ' C45H54N4O16'

We've now finished the 'aerobic' branch, and so I will commit this into the repo before continuing here.

In [40]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

The next step in the proposed hybrid pathway would be the integration of cobalt into the molecule. Beata proposes that this happens in the similar fashion as in S. enterica: with a sirohydrochlorin cobaltochelatase (EC 4.99.1.3) that can also convert the precorrin-5A. This would convert the Precorrin5'A' into cobalt-precorrin5A. 

I will add this reaction into the model.

Also, in the model currently, the reaction carries by CobF is present. However, there is no gene annotation for CobF. To prevent the normal aerobic pathway from running, I will remove the P6AS reaction that is catalyzed by CobF from the model too.

In [50]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [51]:
#add co-precorrin 5 metabolite
model.add_metabolites(Metabolite(id='copre5_c', name = 'Cobalt-precorrin 5', compartment = 'c', charge = 0, formula = 'C45H52CoN4O16'))

In [52]:
model.metabolites.copre5_c.annotation['sbo']='SBO:0000247'
model.metabolites.copre5_c.annotation['kegg.compound']='C16242'
model.metabolites.copre5_c.annotation['chebi']='CHEBI:52488'

In [53]:
#add reaction
model.add_reaction(Reaction(id='SHCLCC', name = 'sirohydrochlorin cobaltochelatase'))

In [54]:
model.reactions.SHCLCC.annotation['sbo']= 'SBO:0000176'
model.reactions.SHCLCC.annotation['ec-code']= '4.99.1.3'

In [55]:
model.groups.get_by_id('00860 - Porphyrin and chlorophyll metabolism').add_members(model.reactions.SHCLCC)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



In [56]:
model.reactions.SHCLCC.bounds = (-1000,1000)

In [57]:
model.reactions.SHCLCC.add_metabolites({
    model.metabolites.pre5a_c:-1,
    model.metabolites.cobalt2_c:-1,
    model.metabolites.copre5_c:1,
    model.metabolites.h_c:2
})

In [58]:
model.reactions.SHCLCC.check_mass_balance()

{}

In [59]:
model.remove_reactions(model.reactions.P6AS)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [60]:
#save and commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now I've dealt with the upstream part of the hybrid pathway, and the cobalt insertion. The next step is to then finish the anaerobic part of the pathway down until cobyrinate-a,c-diamide.

#need to check that the model actually uses this pathway
so delete steps from the pathways that are not annotated and or used! 
    
- e.g. delete cobF as it isnt annotated to prevent complete aerobic pathway
remove CbiT and CbiE